In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from embedders import Embedder
from database.milvusdb import MilvusDB
from query_expander import get_expander

tqdm.pandas()
embedder = Embedder.create("Qwen/Qwen3-Embedding-0.6B", device="mps", normalize=True, for_queries=True)

In [2]:
QUERY_EXPANSION = 'add_prev_3'

# from compute_difference_vector import get_sample_df
# samples = get_sample_df(14000, embedder)

In [ ]:
examples = pd.read_json("data/dataset/nontrivial_checked.jsonl", lines=True)
expander = get_expander(QUERY_EXPANSION, path_to_data="data/preprocessed/reviews.jsonl")
print(f"Using query expansion: {expander}")
examples["sent_no_cit"] = expander(examples)

# Add vector column to examples
examples["vector"] = examples.progress_apply(lambda row: embedder([row["sent_no_cit"]])[0], axis=1)

# Denormalize on citation_dois (targets)
examples = examples.explode("citation_dois", ignore_index=True)
print(f"Number of samples after denormalization: {examples.shape[0]}")
examples.rename(columns={"citation_dois": "target_doi"}, inplace=True)

Using query expansion: QueryExpander(name=add_prev_3, data_length=2980)


 43%|████▎     | 6367/14735 [08:56<23:18,  5.98it/s]  

In [ ]:
examples.head()

In [ ]:
db = MilvusDB()
def most_similar_to_query(example: pd.Series) -> np.ndarray:
    """
    Takes in an example (with 'vector' column already set), and from the candidates
    (returned entities with that doi from the database), returns the vector most similar
    to the example's vector.

    """
    # Converts 'vector' column to rows * dim array, holding the candidate vectors
    candidates = db.select_by_doi(example.target_doi, collection_name="qwen06_chunks")
    candidate_vectors = np.stack(candidates["vector"])
    best_idx = np.argmax(np.dot(candidate_vectors, example["vector"]))
    best_vector = candidate_vectors[best_idx]
    return best_vector

In [ ]:
Q = np.array(samples['vector'].tolist()).T

V = np.array([most_similar_to_query(row) for _, row in tqdm(samples.iterrows(), total=len(samples))]).T
print("Q shape:", Q.shape)
print("V shape:", V.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

M = V @ Q.T
plt.figure(figsize=(9, 7))
sns.heatmap(M, cmap="coolwarm", center=0, cbar_kws={"label": "value"})  # center=0 for diverging data
plt.title("Matrix heatmap (seaborn)")
plt.tight_layout()
plt.savefig("heatmap_seaborn.png", dpi=200)
plt.show()

In [ ]:

U, S, Vt = np.linalg.svd(M)
R = U @ Vt

print("det R:", np.linalg.det(R))
# if np.linalg.det(R) < 0:
#     D = np.eye(R.shape[0])
#     D[-1, -1] = -1
#     R = U @ D @ Vt
# print("corrected det R:", np.linalg.det(R))

In [ ]:
research_dois = set(samples.target_doi)
print(research_dois)

In [ ]:
import random

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def negative_vectors(example: pd.Series, n=5):
    # Get all target DOIs for this example, sample from those not cited
    rows = samples[(samples.source_doi == example.source_doi) & (samples.sent_idx == example.sent_idx)]
    citation_dois = set(rows.target_doi)
    neg_dois = list(research_dois - citation_dois)
    neg_samples = random.sample(neg_dois, n)
    
    # Get vectors for negative samples
    neg_vectors = np.zeros((n, embedder.dim))
    for i, doi in enumerate(neg_samples):
        records = db.select_by_doi(doi, collection_name="qwen06_chunks")
        sample_record = records.sample(n=1).iloc[0]
        neg_vectors[i] = np.array(sample_record['vector'])
    return neg_vectors


res = negative_vectors(samples.iloc[998])
print(res)

Citation DOIs: {'10.1046/j.1365-8711.2000.03810.x', '10.1086/316394', '10.1086/186883'}

In [ ]:
df = db.select_by_doi("10.1046/j.1365-8711.2000.03810.x", collection_name="qwen06_chunks")
row = df.sample(n=1).iloc[0]
print(np.array(row['vector']))

In [ ]:
differences_to_target = []
differences_to_negative = []
for _, row in samples.iterrows():
    query_vector = row['vector']
    target_vector = most_similar_to_query(row)
    aligned_query_vector = R @ query_vector
    before = cosine_similarity(query_vector, target_vector)
    after = cosine_similarity(aligned_query_vector, target_vector)

    differences_to_target.append(after - before)
    neg_vectors = negative_vectors(row, n=5)
    batch_distance_to_negative = []
    for neg_vector in neg_vectors:
        before_neg = cosine_similarity(query_vector, neg_vector)
        after_neg = cosine_similarity(aligned_query_vector, neg_vector)
        batch_distance_to_negative.append(after_neg - before_neg)
        differences_to_negative.append(after_neg - before_neg)
    print(_)
    print(f"Improvement to target: {after - before:.4f}")
    print(f"Distance to negatives: {np.mean(batch_distance_to_negative):.4f} ± {np.std(batch_distance_to_negative):.4f}")
    print("---")
print(f"Average improvement (target): {np.mean(differences_to_target):.6f} ± {np.std(differences_to_target):.6f}")
print(f"Average improvement (negative): {np.mean(differences_to_negative):.6f} ± {np.std(differences_to_negative):.6f}")

In [ ]:
print(R.shape)

In [ ]:

U, S, Vt = np.linalg.svd(M, full_matrices=False)
print("rank approx:", np.sum(S > 1e-8))
print("singular values (top/last):", S[:30], S[-5:])

In [ ]:
import matplotlib.pyplot as plt
U, S, Vt = np.linalg.svd(M)
print("U shape:", U.shape)
print("S shape:", S.shape)
print("Vt shape:", Vt.shape)
print("Vt_trunc shape:", Vt[:1, :].shape)
print("S_trunc shape:", S[:1].shape)
print("U_trunc shape:", U[:, 0:1].shape)
trunc = U[:, 0:1] @ (S[:1] * Vt[0:1, :])
print("trunc shape:", trunc.shape)
print("trunc rank:", np.linalg.matrix_rank(trunc))
print(trunc)


plt.semilogy(S[:20])  # plot singular values

plt.figure(figsize=(9, 7))
sns.heatmap(trunc, cmap="coolwarm", center=0, cbar_kws={"label": "value"})  # center=0 for diverging data
plt.title("Matrix heatmap (seaborn)")
plt.tight_layout()
plt.show()

In [ ]:
np.save('qwen06_chunks_rotation_n2000.npy', R)

In [ ]:
sigma1 = S[0]
eps = 1e-6  # try 1e-6 .. 1e-8 as needed
r = 50
# r = np.searchsorted(S / sigma1 < eps, True)  # first index where ratio < eps
print(f"Chosen rank r: {r}")
if r == 0:
    r = len(S)  # fallback if no small ones found
# alternative: r = np.searchsorted(np.cumsum(S**2) / np.sum(S**2), 0.99) + 1

# build small rotation in top-r
Ur = U[:, :r]  # n x r
Vr = Vt[:r, :].T  # n x r  (since Vt[:r,:] is r x n)
Msmall = Ur.T @ M @ Vr  # should be r x r but simpler compute: Tproj.T @ Qproj if you had them
# simpler: compute r x r cross-covariance directly via projections:
# Qproj = Q_all @ Ur  # expensive if many, but doable; here we reuse M decomposition
# but we can use SVD of the small Mslice: compute Us, Ss, Vts = svd(Ur.T @ M @ Vr)

# directly SVD the r x r matrix (numerically stable):
Us, Ss, Vts = np.linalg.svd(Ur.T @ M @ Vr, full_matrices=False)
Rsmall = Us @ Vts
# ensure proper rotation (det +1)
if np.linalg.det(Rsmall) < 0:
    D = np.eye(r)
    D[-1, -1] = -1
    Rsmall = Us @ D @ Vts

# map basis Ur to rotated basis Ur @ Rsmall @ Ur.T, then add identity on complement
R_full = Ur @ Rsmall @ Ur.T + (np.eye(1024) - Ur @ Ur.T)
print("det R_full:", np.linalg.det(R_full))
print(R_full.shape)
print(R_full)

In [ ]:
np.save('qwen06_chunks_r50.npy', R_full)